In [1]:
import pandas as pd
import numpy as np
import yaml
import gower
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df = pd.read_csv(config['output_data']['file'])

In [11]:
def body_composition_analysis(df):
    gender = input("Gender (Male/Female): ").strip().title()
    age = int(input("Age (years): "))
    height_m = float(input("Height (in meters, e.g., 1.70): "))
    weight_kg = float(input("Weight (in kg(40- 130): "))
    diet_type = input("Enter your Diet Type (Balanced, Keto, Low-Carb, Paleo, Vegan, Vegetarian): ").strip().title()
    workout_type = input("Enter your Workout Type (Cardio, Hiit, Strength, Yoga): ").strip().title()
    avg_bpm = float(input("Enter your average workout BPM (e.g., 119–170): ") or 140)
    calories_burned = float(input("Calories burned per session (e.g., 320–2891): ") or 800)
    water_intake_liters = float(input("Daily water intake in liters (e.g., 1,2,3): ") or 2)


    # --- Gower Distance Similarity (Top 3 Neighbors) ---
    cols_for_match = [
        "gender", "age", "height_m", "weight_kg",
        "diet_type", "workout_type", "avg_bpm",
        "calories_burned", "water_intake_liters"
    ]
    cols_for_match = [c for c in cols_for_match if c in df.columns]

    input_data = pd.DataFrame([{
        "gender": gender,
        "age": age,
        "height_m": height_m,
        "weight_kg": weight_kg,
        "diet_type": diet_type,
        "workout_type": workout_type,
        "avg_bpm": avg_bpm,
        "calories_burned": calories_burned,
        "water_intake_liters": water_intake_liters
    }])

    df_subset = df[cols_for_match].dropna().copy()
    gower_distances = gower.gower_matrix(df_subset, input_data)
    gower_flat = gower_distances.flatten()

    top_k = 3
    topk_idx = np.argsort(gower_flat)[:top_k]
    matched_rows = df.iloc[topk_idx]
    matched_row = matched_rows.mean(numeric_only=True)

    print(f"\n Top {top_k} similar profiles found:")
    print(matched_rows[["age", "height_m", "weight_kg"]].head())

    # --- Derived Metrics ---
    bmi = weight_kg / (height_m ** 2)
    gender_factor = 1 if gender.lower() == "male" else 0
    body_fat_percentage = (1.20 * bmi) + (0.23 * age) - (10.8 * gender_factor) - 5.4
    fat_mass_kg = weight_kg * (body_fat_percentage / 100)
    lean_mass_kg = weight_kg - fat_mass_kg

    calories = matched_row.get("calories", 0)
    calories_burned = matched_row.get("calories_burned", 0)
    proteins = matched_row.get("proteins", 0)
    carbs = matched_row.get("carbs", 0)
    fats = matched_row.get("fats", 0)
    session_duration_hours = matched_row.get("session_duration_hours", 1)
    water_intake_liters = matched_row.get("water_intake_liters", 2)
    max_bpm = matched_row.get("max_bpm", 180)
    avg_bpm = matched_row.get("avg_bpm", 140)
    resting_bpm = matched_row.get("resting_bpm", 70)

    total_calories = (carbs * 4) + (proteins * 4) + (fats * 9)
    pct_carbs = (carbs * 4 / total_calories) * 100 if total_calories > 0 else 0
    protein_per_kg = proteins / weight_kg if weight_kg > 0 else 0
    cal_balance = calories - calories_burned
    workout_efficiency = calories_burned / session_duration_hours if session_duration_hours > 0 else 0
    hr_reserve = max_bpm - resting_bpm
    hydration_score = water_intake_liters / (weight_kg * 0.033)
    nutrition_quality = (
        (0.4 * protein_per_kg)
        + (0.3 * (100 - abs(pct_carbs - 50)) / 100)
        + (0.3 * hydration_score)
    )

    # --- BMI Classification ---
    if bmi < 18.5:
        bmi_category, bmi_reco = "Underweight", "Increase calorie intake and focus on nutrient-rich foods."
    elif 18.5 <= bmi < 25:
        bmi_category, bmi_reco = "Normal weight", "Maintain your current healthy habits."
    elif 25 <= bmi < 30:
        bmi_category, bmi_reco = "Overweight", "Increase activity levels and moderate calorie intake."
    elif 30 <= bmi < 40:
        bmi_category, bmi_reco = "Obese", "Adopt a structured exercise and diet plan."
    else:
        bmi_category, bmi_reco = "Severely Obese", "Seek professional medical guidance."

    # --- Body Fat % Classification ---
    if gender.lower() == "male":
        if body_fat_percentage < 6: fat_category = "Essential Fat"
        elif 6 <= body_fat_percentage < 14: fat_category = "Athlete"
        elif 14 <= body_fat_percentage < 18: fat_category = "Fit"
        elif 18 <= body_fat_percentage < 25: fat_category = "Average"
        else: fat_category = "Obese"
    else:
        if body_fat_percentage < 14: fat_category = "Essential Fat"
        elif 14 <= body_fat_percentage < 21: fat_category = "Athlete"
        elif 21 <= body_fat_percentage < 25: fat_category = "Fit"
        elif 25 <= body_fat_percentage < 32: fat_category = "Average"
        else: fat_category = "Obese"

    # --- Summary ---
    if bmi_category == "Normal weight" and fat_category in ["Fit", "Athlete"]:
        summary = "Excellent shape! Keep maintaining a balanced lifestyle."
    elif bmi_category == "Overweight" and fat_category in ["Average", "Obese"]:
        summary = "Moderate excess — increase activity and portion control."
    elif bmi_category in ["Obese", "Severely Obese"]:
        summary = "High body composition risk — start a sustainable calorie deficit and stay active."
    elif bmi_category == "Underweight":
        summary = "Low BMI — increase muscle mass with resistance training and higher protein intake."
    else:
        summary = "You're close to a healthy range — maintain balanced nutrition and regular exercise."

    # --- Display Results ---
    metrics_df = pd.DataFrame({
        "Metric": [
            "BMI", "BMI Category", "Body Fat %", "Body Fat Category",
            "Lean Mass (kg)", "Protein per kg", "Workout Efficiency (kcal/hr)",
            "Calorie Balance (kcal)", "Hydration Score"],
        "Value": [
            round(bmi, 2), bmi_category, round(body_fat_percentage, 2), fat_category,
            round(lean_mass_kg, 2), round(protein_per_kg, 2), round(workout_efficiency, 0),
            round(cal_balance, 0), round(hydration_score, 2)]
    })

    print(metrics_df.to_string(index=False))
    print("\n Summary:", summary)
    print("BMI Insight:", bmi_reco)
    print(" Body Fat Insight:", fat_category)

    # Additional Nudges
    if hydration_score < 0.8:
        print(" You may be under-hydrated. Aim for ~35 ml water per kg body weight daily.")
    if protein_per_kg < 1.2:
        print(" Increase protein intake to support muscle recovery.")
    if cal_balance < -300:
        print(" You're in a calorie deficit — good for fat loss if intentional.")
    elif cal_balance > 300:
        print(" Calorie surplus detected — could aid muscle gain or cause fat gain.")

    return metrics_df, summary


if __name__ == "__main__":
    body_composition_analysis(df)


Gender (Male/Female):  Male
Age (years):  34
Height (in meters, e.g., 1.70):  1.65
Weight (in kg(40- 130):  78
Enter your Diet Type (Balanced, Keto, Low-Carb, Paleo, Vegan, Vegetarian):  Vegan
Enter your Workout Type (Cardio, Hiit, Strength, Yoga):  Hiit
Enter your average workout BPM (e.g., 119–170):  160
Calories burned per session (e.g., 320–2891):  700
Daily water intake in liters (e.g., 1,2,3):  2



 Top 3 similar profiles found:
       age  height_m  weight_kg
7158    28      1.62         76
943     47      1.69         68
13173   30      1.62         78
                      Metric      Value
                         BMI      28.65
                BMI Category Overweight
                  Body Fat %       26.0
           Body Fat Category      Obese
              Lean Mass (kg)      57.72
              Protein per kg       1.12
Workout Efficiency (kcal/hr)     1292.0
      Calorie Balance (kcal)      642.0
             Hydration Score       0.78

 Summary: Moderate excess — increase activity and portion control.
BMI Insight: Increase activity levels and moderate calorie intake.
 Body Fat Insight: Obese
 You may be under-hydrated. Aim for ~35 ml water per kg body weight daily.
 Increase protein intake to support muscle recovery.
 Calorie surplus detected — could aid muscle gain or cause fat gain.
